<a href="https://colab.research.google.com/github/JManuelRG/propiedades_termodinamicas/blob/main/Sustancias%20Puras/Ecuacion_de_Antoine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ecuación de Antoine
# Contantes Obtenidas de la base de datos web de NIST:

https://webbook.nist.gov/chemistry/

# Web scrapper:
https://github.com/oscarcontrerasnavas/NIST-web-book-scraping/blob/master/README.md

https://pypi.org/project/beautifulsoup4/


# Instrucciones:

Ejecutar la siguiente función para obtener los parámetros de la ecuación de Antoine.

La celda es un Web Scrapper que buscará el compuesto en la base de datos de NIST y obtendrá los datos para ese compuesto.

In [17]:
# Importar la función get del módulo requests porque es la función encargada de
# hacer la solicitud HTTP GET con la URL proporcionada.
from requests import get

# Importar BeautifulSoup de bs4 porque hace el análisis HTML y nos ayuda a
# manejar el DOM.
from bs4 import BeautifulSoup

# Importar closing para asegurar que cualquier recurso de red se libere cuando
# salga de alcance.
from contextlib import closing

def get_antoine_coef(Nombre, Temperatura):

    """ Devuelve una lista con los coeficientes A, B y C si existen para la
        Temperatura dada. Si no, retorna None e imprime un mensaje.

    :param Nombre:
        Una cadena con el nombre del compuesto en inglés.

    :param Temperatura:
        Un número flotante con la temperatura en Kelvin.

    :rtype: Lista

    :return coef con [A, B, C]

    """

    # Obteniendo la tabla usando la función get_html mostrada abajo. Table es un
    # Objeto BeautifulSoup.
    tabla = get_html_table(Nombre)

    # Extraer las filas de la tabla. Sabiendo qué etiquetas tiene una tabla HTML.
    # Además, sabiendo que la primera fila con el encabezado de la tabla no tiene el
    # atributo de clase 'exp' así que obtenemos solo las filas con datos.
    # La función find_all de BeautifulSoup devuelve una lista.
    filas = tabla.find_all('tr', class_='exp')

    # Declarar las listas para almacenar Temperaturas y coeficientes.
    Temperaturas, As, Bs, Cs = [], [], [], []

    # Iterando sobre las filas para extraer y llenar las variables As, Bs y Cs porque ahora
    # estamos seguros de que las Temperaturas están entre algún rango.
    for fila in filas:

        # Como las filas, extraemos las columnas para la fila actual. Sabiendo que
        # las columnas tienen la etiqueta <td> en HTML también.
        # La función find_all de BeautifulSoup devuelve una lista.
        cols =  fila.find_all('td')

        # Primero transformamos las cadenas en números flotantes y las colocamos en su
        # respectiva lista.
        As.append(float(cols[1].text))
        Bs.append(float(cols[2].text))
        Cs.append(float(cols[3].text))

        # Para las temperaturas, tenemos un rango y necesitamos extraer cada
        # límite (inferior y superior) y colocarlos en una lista adicional. Así que
        # la variable Temperatures será una lista de listas.
        limite_inferior = float(cols[0].text.replace(" ","").split('-')[0])
        limite_superior = float(cols[0].text.replace(" ","").split('-')[1])
        Temperaturas.append([limite_inferior, limite_superior])


    # Verificar si la Temperatura dada se encuentra en algún intervalo
    indice = None
    for i, intervalo in enumerate(Temperaturas):
        if (intervalo[0] <= Temperatura
            and Temperatura <= intervalo[1]):
            indice = i
            break
        else:
            indice = None

    if indice == None:
        print('Lo siento, los datos para la temperatura dada %.2f K no existen en la Base de Datos' % Temperatura)
        return None
    else:
        A = As[indice]
        B = Bs[indice]
        C = Cs[indice]
        return [A, B, C]


def get_html_table(Nombre):

    """ Retorna el HTML ya parseado usando una función auxiliar listada abajo.

    :param Nombre:
        Una cadena con el nombre del compuesto en inglés.

    :rtype: Objeto BeautifulSoup

    """

    # El parámetro name es parte de la URL. Por ejemplo, si deseas los
    # datos de metano, la URL es
    # https://webbook.nist.gov/cgi/cbook.cgi?Name=methane&Mask=4.
    url = str.format('https://webbook.nist.gov/cgi/cbook.cgi?Name={0}&Mask=4', Nombre.lower())

    # Función para obtener la solicitud realizada, ver más abajo.
    html_bruto = get_response(url)

    # Parsear el HTML usando BeautifulSoup.
    html = BeautifulSoup(html_bruto, 'html.parser')

    # Extraer la tabla que contiene los datos, la tabla tiene un
    # atributo específico 'aria-label' como 'Antoine Equation Parameters'.
    tabla = html.find('table', attrs={'aria-label': 'Antoine Equation Parameters'})

    return tabla


def get_response(url):

    """ Retorna el HTML sin procesar para analizarlo más tarde o None si no se puede acceder a la página.

    :param url:
        La cadena para la solicitud GET.

    :rtype: Objeto BeautifulSoup

    :rtype: None si no se puede acceder al sitio web

    """

    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except:
        print('No encontrado')
        return None


def is_good_response(resp):
    """
    Retorna True si la respuesta parece ser HTML, False en caso contrario.
    """
    tipo_contenido = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200
            and tipo_contenido is not None
            and tipo_contenido.find('html') > -1)


# Forma de la ec. de Antoine

$$log_{10}(P) = A −\frac{B}{T+C}$$

P = Presión de vapor (bar),  T = temperatura (K)

In [18]:
import pandas as pd
# Obtener las variables de la ecuación de Antoine
print("Constantes de la Ec. de Antoine:")
nombre = input('Escribe el nombre del compuesto en inglés: ')
T = float(input('Escribe la temperatura promedio del compuesto, K: '))
A,B,C = get_antoine_coef(nombre,
                  T)
print(f'Tabla de constantes de Antoine para -{nombre}-, válidos a una temperatura de {T} Kelvin')
print('Obtenidos de la base de Datos NIST:')
print(f'URL = https://webbook.nist.gov/cgi/cbook.cgi?Name={nombre}&Mask=4')
datosAntoine = pd.DataFrame({'Compuesto':nombre,
                             'A':[A],
                             'B':[B],
                             'C':[C]})
datosAntoine.set_index('Compuesto')

Constantes de la Ec. de Antoine:
Escribe el nombre del compuesto en inglés: water
Escribe la temperatura promedio del compuesto, K: 350
Tabla de constantes de Antoine para -water-, válidos a una temperatura de 350.0 Kelvin
Obtenidos de la base de Datos NIST:
URL = https://webbook.nist.gov/cgi/cbook.cgi?Name=water&Mask=4


,A,B,C
Compuesto,,,
water,5.0768,1659.793,-45.854


# Valores Puntuales:

# Cálculo de la Presión de saturación de la sustancia:
# Forma de la ec. de Antoine

$$log_{10}(P) = A −\frac{B}{T+C}$$

P = Presión de vapor (bar),  T = temperatura (K)

$$P^{sat} = 10^{A −\frac{B}{T^{sat}+C}}$$

In [6]:
print('Determinación de la Presión de saturación de un sistema:')
Tsat = float(input('Escribe el valor de la temperatura de saturación en °C: '))
Psat = 10**(datosAntoine['A']-datosAntoine['B']/(Tsat + 273.15 + datosAntoine['C']))
print(f"La presión de saturación de '{datosAntoine['Compuesto'][0]}' calculada, es de {Psat[0]} bar")
print(f"Psat = {Psat[0]*100} kPa")

Determinación de la Presión de saturación de un sistema:
Escribe el valor de la temperatura de saturación en °C: 75
La presión de saturación de 'water' calculada, es de 0.3856365823351287 bar
Psat = 38.56365823351287 kPa


# Cálculo de la Temperatura de Saturación de la sustancia:

$$T(°C) = \frac{B}{(A-log_{10}P(bar))}-C$$

In [23]:
import math
print('Determinación de la temperatura de saturación de un sistema:')
Psatc = float(input('Escribe el valor de la presión de saturación en bar: '))

Tsatc = datosAntoine['B']/(datosAntoine['A']-math.log10(Psatc))-datosAntoine['C']

print(f"Latemperatura de saturación de {datosAntoine['Compuesto'][0]}, calculada es de {Psatc} bar")
print(f"Tsat = {Tsatc[0]-273.15} °C")


Determinación de la temperatura de saturación de un sistema:
Escribe el valor de la presión de saturación en bar: .4
Latemperatura de saturación de water, calculada es de 0.4 bar
Tsat = 75.8769355861416 °C


In [22]:
Psatc

0.4